In [1]:
!pip install tensorflow

In [2]:
!pip install tensorflow-gpu

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
    Running setup.py install for tensorflow-gpu: started
    Running setup.py install for tensorflow-gpu: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\prajw\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\prajw\\AppData\\Local\\Temp\\pip-install-9gfsk_o8\\tensorflow-gpu_9535929c346a442fa4d679a1184090d3\\setup.py'"'"'; __file__='"'"'C:\\Users\\prajw\\AppData\\Local\\Temp\\pip-install-9gfsk_o8\\tensorflow-gpu_9535929c346a442fa4d679a1184090d3\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\prajw\AppData\Local\Temp\pip-wheel-yikux6d_'
       cwd: C:\Users\prajw\AppData\Local\Temp\pip-install-9gfsk_o8\tensorflow-gpu_9535929c346a442fa4d679a1184090d3\
  Complete output (17 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "C:\Us

In [3]:
!pip install opencv-python

In [4]:
!pip install matplotlib

In [5]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
anaconda-client               1.9.0
anaconda-navigator            2.3.2
anaconda-project              0.10.2
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.6.6
astropy                       5.0.4
asttokens                     2.0.5
astunparse                    1.6.3
async-timeout                 4.0.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backpor

prometheus-client             0.13.1
prompt-toolkit                3.0.20
Protego                       0.1.16
protobuf                      3.19.1
psutil                        5.8.0
ptyprocess                    0.7.0
pure-eval                     0.2.2
py                            1.11.0
pyasn1                        0.4.8
pyasn1-modules                0.2.8
pycodestyle                   2.7.0
pycosat                       0.6.3
pycparser                     2.21
pyct                          0.4.6
pycurl                        7.44.1
PyDispatcher                  2.0.5
pydocstyle                    6.1.1
pyerfa                        2.0.0
pyflakes                      2.3.1
Pygments                      2.11.2
PyHamcrest                    2.0.2
PyJWT                         2.1.0
pylint                        2.9.6
pyls-spyder                   0.4.0
PyNaCl                        1.4.0
pyodbc                        4.0.32
pyOpenSSL                     21.0.0
pyparsing           

In [12]:
!pip install numpy==1.20.0

  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.


In [6]:
import os
import cv2
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt

In [7]:
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [8]:
gpus

[]

In [9]:
def load_image(x):
    byte_img=tf.io.read_file(x)
    img=tf.io.decode_jpeg(byte_img)
    return img

In [ ]:
train_images=tf.data.Dataset.list_files('*.jpg',shuffle=False)
train_images=train_images.map(load_image)
train_images=train_images.map(lambda x:tf.image.resize(x,(250,250)))
train_images=train_images.map(lambda x: x/255)

In [ ]:
test_images=tf.data.Dataset.list_files('*.jpg',shuffle=False)
test_images=test_images.map(load_image)
test_images=test_images.map(lambda x:tf.image.resize(x,(250,250)))
test_images=test_images.map(lambda x: x/255)

In [ ]:
val_images=tf.data.Dataset.list_files('*.jpg',shuffle=False)
val_images=val_images.map(load_image)
val_images=val_images.map(lambda x:tf.image.resize(x,(250,250)))
val_images=val_images.map(lambda x: x/255)

In [ ]:
plt.imshow(train_images.as_numpy_iterator().next())
plt.imshow(test_images.as_numpy_iterator().next())
plt.imshow(val_images.as_numpy_iterator().next())

In [ ]:
def load_label(label_path):
    with open(label_path.numpy(),'r',encoding="utf-8") as f:
        label = json.load(f)
    return [label['keypoints']]

In [ ]:
train_label=tf.data.Dataset.list_files('*.json',shuffle=False)
train_label=train_label.map(lambda x:tf.py_function(load_label,[x],[tf.float16]))

In [ ]:
test_label=tf.data.Dataset.list_files('*.json',shuffle=False)
test_label=test_label.map(lambda x:tf.py_function(load_label,[x],[tf.float16]))

In [ ]:
val_label=tf.data.Dataset.list_files('*.json',shuffle=False)
val_label=val_label.map(lambda x:tf.py_function(load_label,[x],[tf.float16]))

### First Two X - Left Eye; Y - Left Eye
### Second Two X - Right Eye; Y - Right Eye

Combine Image and label

In [ ]:
train = tf.data.Dataset.zip((train_images,train_label))
train = train.shuffle(8000)
train = train.batch(16)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((train_images,train_label))
test = test.shuffle(1300)
test = test.batch(16)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((train_images,train_label))
val = val.shuffle(200)
val = val.batch(16)
val = val.prefetch(4)

View Samples

In [ ]:
data_samples = train.as_numpy_iterator()

In [ ]:
res = data_samples.next()

In [ ]:
fig,ax=plt.subplots(ncols=4,figsize=(20,20))
for idx in range(4):
    sample_image=res[0][idx]
    sample_coords=res[1][0][idx]
    cv2.circle(sample_image,tuple(np.multiply(sample_coords[:2],[250,250]).astype(int)),2, (255,0,0),-1)
    cv2.circle(sample_image,tuple(np.multiply(sample_coords[2:],[250,250]).astype(int)),2, (0,255,0),-1)
    ax[idx].imshow(sample_image)